In [ ]:
# SENTIMENT ANALYSIS MULTI-SCHEME TRAINING

# Cell 1: Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import re

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

nltk.download('punkt')
nltk.download('stopwords')


AttributeError: module 'numpy' has no attribute 'dtypes'

In [ ]:
!pip install transformers --quiet
from transformers import pipeline
import pandas as pd

# Load dataset
df = pd.read_csv('scrapped_comments.csv')
df.dropna(inplace=True)

# Load sentiment analysis pipeline
sentiment_pipeline = pipeline("text-classification", model="mdhugol/indonesia-bert-sentiment-classification")

# Mapping label uppercase ke bentuk teks
label_mapping = {
    'LABEL_0': 'negatif',
    'LABEL_1': 'netral',
    'LABEL_2': 'positif'
}

# Terapkan ke kolom komentar
def get_label(text):
    result = sentiment_pipeline(text[:512])[0]  # Batasin biar gak lebih dari 512 token
    return label_mapping[result['label']]

# Buat label otomatis
df['label'] = df['comments'].apply(get_label)

# Pilih hanya kolom yang dibutuhkan
df = df[['comments', 'label']]

# Tampilkan 25 hasil pertama
from IPython.display import display
display(df.head(25))

# Simpan ke file CSV
df.to_csv('labeled_comments.csv', index=False)
print("File saved as labeled_comments.csv ✅")


Device set to use cpu


,comments,label
0,ayo nitizen be smart jng lg dibodohin kuat gel...,negatif
1,masyarakat indonesia banyak edukasi,negatif
2,gorong tikus yg rakus liar,positif
3,gak paling tiada tanding,negatif
4,gak dua,negatif
5,moga rakyat indonesia video sadar sebener nya ...,positif
6,sobakso wkwk,positif
7,podcast jugayg escapethe end,netral
8,suara rakyat dpr suara rakyat influencer,netral
9,si botak motong bicara mulu gaje,positif


File saved as labeled_comments.csv ✅


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# TF-IDF
tfidf = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf.fit_transform(df['comments']).toarray()

# Encode label
le = LabelEncoder()
y_encoded = le.fit_transform(df['label'])

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y_encoded, test_size=0.2, random_state=42)

# Train SVM
svm_model = SVC(kernel='linear', C=1)
svm_model.fit(X_train, y_train)

# Evaluation
svm_pred = svm_model.predict(X_test)
print("🎯 [SVM + TF-IDF] Accuracy:", accuracy_score(y_test, svm_pred))
print(classification_report(y_test, svm_pred, target_names=le.classes_))


NameError: name 'df' is not defined

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Tokenization
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(df['comments'])
sequences = tokenizer.texts_to_sequences(df['comments'])
X_seq = pad_sequences(sequences, maxlen=100)

# One-hot encode label
y_onehot = pd.get_dummies(df['label']).values

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_onehot, test_size=0.3, random_state=42)

# LSTM model
model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=100),
    LSTM(128, return_sequences=True),
    Dropout(0.5),
    LSTM(64),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train
history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))

# Evaluation
train_loss, train_acc = model.evaluate(X_train, y_train)
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"🎯 [LSTM] Train Accuracy: {train_acc:.4f} | Test Accuracy: {test_acc:.4f}")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.4400 - loss: 1.0954 - val_accuracy: 0.3667 - val_loss: 1.0795
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 295ms/step - accuracy: 0.4648 - loss: 1.0523 - val_accuracy: 0.3667 - val_loss: 1.0853
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 294ms/step - accuracy: 0.4596 - loss: 1.0104 - val_accuracy: 0.3667 - val_loss: 1.2531
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 568ms/step - accuracy: 0.4648 - loss: 1.0808 - val_accuracy: 0.3667 - val_loss: 1.3771
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 392ms/step - accuracy: 0.4648 - loss: 1.1428 - val_accuracy: 0.3667 - val_loss: 1.2948
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 0.4814 - loss: 1.0827 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - accuracy: 0.3667 - loss: 1.2948
🎯 [LSTM] Train Accuracy: 0.4706 | Test Accuracy: 0.3667
